# Notebook 4: LQR Validation\n\n**Author:** Divyansh Atri\n\n## Overview\n\nValidate numerical HJB solver against analytical LQR solution.\n\nThe Linear-Quadratic Regulator has a known analytical solution via the Riccati equation, making it perfect for validation.

In [ ]:
import numpy as np\nimport matplotlib.pyplot as plt\nimport sys\nsys.path.append('..')\nfrom utils import *\n\nplt.style.use('seaborn-v0_8-darkgrid')\nplt.rcParams['figure.figsize'] = (14, 6)\n\nprint('LQR Validation - Ready')

## 1. LQR Problem Setup\n\n**Dynamics:**\n$$dX_t = (AX_t + Bu_t) dt + \sigma dW_t$$\n\n**Cost:**\n$$J = \mathbb{E}\left[\int_0^T \frac{1}{2}(qX_t^2 + ru_t^2) dt + \frac{1}{2}q_T X_T^2\right]$$\n\n**Analytical Solution:**\n$$V(t,x) = \frac{1}{2}P(t)x^2 + \psi(t)$$\n\nwhere $P(t)$ solves the Riccati ODE:\n$$\frac{dP}{dt} = -q + 2AP - \frac{B^2}{r}P^2, \quad P(T) = q_T$$

In [ ]:
# Parameters\nA, B, sigma = -1.0, 1.0, 0.5\nq, r, q_T = 1.0, 1.0, 10.0\nT = 2.0\n\nprint(f'LQR Parameters:')\nprint(f'  Dynamics: A={A}, B={B}, σ={sigma}')\nprint(f'  Cost: q={q}, r={r}, q_T={q_T}')\nprint(f'  Time horizon: T={T}')

## 2. Analytical Solution

In [ ]:
# Solve Riccati equation\nnt = 201\nt = np.linspace(0, T, nt)\nx = np.linspace(-3, 3, 101)\n\nV_analytical, P = lqr_analytical_solution(A, B, sigma, q, r, q_T, T, t, x)\n\nprint(f'Analytical solution computed')\nprint(f'P(0) = {P[0]:.6f}')\nprint(f'P(T) = {P[-1]:.6f} (should be {q_T})')\nprint(f'V(0, 0) = {V_analytical[0, len(x)//2]:.6f}')

## 3. Numerical Solution

In [ ]:
# Setup numerical solver\nmodel = LinearQuadraticModel(A=A, B=B, sigma=sigma)\ncost_fn = QuadraticCost(q=q, r=r, q_terminal=q_T)\n\nsolver = HJBSolver(-3.0, 3.0, 101, T, nt, model, cost_fn)\n\nprint('Solving HJB numerically...')\nV_numerical, u_opt = solver.solve_backward(u_bounds=(-10, 10), verbose=False)\n\nprint(f'Numerical solution computed')\nprint(f'V_num(0, 0) = {V_numerical[0, len(x)//2]:.6f}')

## 4. Comparison

In [ ]:
# Compute error\nerror = np.abs(V_numerical - V_analytical)\nmax_error = np.max(error)\nmean_error = np.mean(error)\nrel_error = max_error / np.max(np.abs(V_analytical))\n\nprint(f'\nError Analysis:')\nprint(f'  Max absolute error: {max_error:.6e}')\nprint(f'  Mean absolute error: {mean_error:.6e}')\nprint(f'  Relative error: {rel_error:.6e}')\n\n# Plot comparison\nfig, axes = plt.subplots(2, 2, figsize=(14, 10))\n\n# Value function at t=0\naxes[0,0].plot(x, V_analytical[0, :], 'b-', linewidth=2, label='Analytical')\naxes[0,0].plot(x, V_numerical[0, :], 'r--', linewidth=2, label='Numerical')\naxes[0,0].set_xlabel('State $x$')\naxes[0,0].set_ylabel('Value $V(0,x)$')\naxes[0,0].set_title('Value Function at $t=0$')\naxes[0,0].legend()\naxes[0,0].grid(True, alpha=0.3)\n\n# Value function at t=T/2\nmid_idx = nt // 2\naxes[0,1].plot(x, V_analytical[mid_idx, :], 'b-', linewidth=2, label='Analytical')\naxes[0,1].plot(x, V_numerical[mid_idx, :], 'r--', linewidth=2, label='Numerical')\naxes[0,1].set_xlabel('State $x$')\naxes[0,1].set_ylabel(f'Value $V({T/2:.1f},x)$')\naxes[0,1].set_title(f'Value Function at $t={T/2:.1f}$')\naxes[0,1].legend()\naxes[0,1].grid(True, alpha=0.3)\n\n# Error heatmap\nT_grid, X_grid = np.meshgrid(t, x, indexing='ij')\nim = axes[1,0].contourf(T_grid, X_grid, error, levels=20, cmap='hot')\naxes[1,0].set_xlabel('Time $t$')\naxes[1,0].set_ylabel('State $x$')\naxes[1,0].set_title('Absolute Error $|V_{num} - V_{ana}|$')\nplt.colorbar(im, ax=axes[1,0])\n\n# Riccati solution P(t)\naxes[1,1].plot(t, P, 'b-', linewidth=2)\naxes[1,1].axhline(q_T, color='r', linestyle='--', label=f'Terminal: $P(T)={q_T}$')\naxes[1,1].set_xlabel('Time $t$')\naxes[1,1].set_ylabel('$P(t)$')\naxes[1,1].set_title('Riccati Solution $P(t)$')\naxes[1,1].legend()\naxes[1,1].grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.savefig('../plots/lqr_validation.png', dpi=150, bbox_inches='tight')\nplt.show()

## 5. Optimal Control Verification

In [ ]:
# Analytical optimal control: u* = -(B/r)P(t)x\nu_analytical = np.zeros((nt, len(x)))\nfor n in range(nt):\n    u_analytical[n, :] = -(B / r) * P[n] * x\n\n# Compare\nu_error = np.abs(u_opt - u_analytical)\n\nplt.figure(figsize=(14, 5))\n\nplt.subplot(1, 2, 1)\nfor idx in [0, nt//4, nt//2, 3*nt//4]:\n    plt.plot(x, u_analytical[idx, :], '-', linewidth=2, label=f't={t[idx]:.2f} (ana)')\n    plt.plot(x, u_opt[idx, :], '--', linewidth=2, alpha=0.7, label=f't={t[idx]:.2f} (num)')\nplt.xlabel('State $x$')\nplt.ylabel('Control $u^*(t,x)$')\nplt.title('Optimal Control: Analytical vs Numerical')\nplt.legend(fontsize=9)\nplt.grid(True, alpha=0.3)\n\nplt.subplot(1, 2, 2)\nim = plt.contourf(T_grid, X_grid, u_error, levels=20, cmap='hot')\nplt.xlabel('Time $t$')\nplt.ylabel('State $x$')\nplt.title('Control Error $|u_{num}^* - u_{ana}^*|$')\nplt.colorbar(im)\n\nplt.tight_layout()\nplt.savefig('../plots/lqr_control_validation.png', dpi=150, bbox_inches='tight')\nplt.show()\n\nprint(f'\nControl Error:')\nprint(f'  Max: {np.max(u_error):.6e}')\nprint(f'  Mean: {np.mean(u_error):.6e}')

## Summary\n\n**Validation Results:**\n- Numerical solution matches analytical solution to high precision\n- Errors are small and consistent with discretization\n- Optimal control policy agrees with analytical formula\n\nThis validates the correctness of our HJB solver.\n\n**Next:** Policy iteration methods.